In [6]:
import subprocess
import os
import pandas as pd

def ucitaj_sekvencu_iz_fajla(fajl_putanja):
    with open(fajl_putanja, 'r') as fajl:
        linije = fajl.readlines()
    return "".join(linija.strip() for linija in linije[1:])

def pokreni_vsl2_na_fajlu(fajl_putanja):
    vsl2_putanja = "/home/user/Desktop/VSL2/VSL2.jar"
    rezultat = subprocess.run(['java', '-jar', vsl2_putanja, fajl_putanja], capture_output=True, text=True)

    if rezultat.stderr:
        print(f"Greška pri pokretanju VSL2: {rezultat.stderr}")
    return parse_vsl2b_output(rezultat.stdout)

def parse_vsl2b_output(vsl2b_output):
    oznake = []
    for line in vsl2b_output.splitlines():
        parts = line.strip().split()
        if len(parts) >= 3:
            try:
                score = float(parts[2])
                oznake.append("uređena" if score <= 0.5 else "neuređena")
            except ValueError:
                continue
    return oznake

def generisi_prozore_i_klasifikuj(sekvenca, min_duzina=30, max_duzina=100):
    podaci = []
    temp_fasta = "temp_prozor.fasta"
    
    for duzina in range(min_duzina, max_duzina + 1):
        for i in range(len(sekvenca) - duzina + 1):
            podsekvenca = sekvenca[i:i + duzina]
            pozicija = f"{i+1}-{i+duzina}"
            
            with open(temp_fasta, 'w') as f:
                f.write(f">prozor_{i}\n{podsekvenca}\n")
            
            oznake = pokreni_vsl2_na_fajlu(temp_fasta)
            oznaka_prozora = "uređena" if oznake.count("uređena") > len(oznake) / 2 else "neuređena"
            
            podaci.append((duzina, pozicija, podsekvenca, oznaka_prozora))
    
    os.remove(temp_fasta)
    return podaci

# Učitavanje sekvence
fajl_putanja = "Spike glycoprotein/YP_138523.1 | spike glycoprotein.fasta"
sekvenca = ucitaj_sekvencu_iz_fajla(fajl_putanja)

# Generisanje prozora i klasifikacija
podaci = generisi_prozore_i_klasifikuj(sekvenca)

# Čuvanje u DataFrame i CSV
kolone = ["Duzina", "Pozicija", "Sekvenca", "VSL2 oznaka"]
df_sekvence = pd.DataFrame(podaci, columns=kolone)

df_sekvence.to_csv("Spike glycoprotein/Klasifikovane_sekvence_VSL2_spike.csv", index=False)
print(df_sekvence.head())


   Duzina Pozicija                        Sekvenca VSL2 oznaka
0      30     1-30  MGGLSLLQLPRDKFRKSSFFVWVIILFQKA   neuređena
1      30     2-31  GGLSLLQLPRDKFRKSSFFVWVIILFQKAF   neuređena
2      30     3-32  GLSLLQLPRDKFRKSSFFVWVIILFQKAFS   neuređena
3      30     4-33  LSLLQLPRDKFRKSSFFVWVIILFQKAFSM   neuređena
4      30     5-34  SLLQLPRDKFRKSSFFVWVIILFQKAFSMP   neuređena
